In [1]:
!pip -q install -U ultralytics opencv-python numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 72.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.4.1 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.1 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.1 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.4.1 which is incompatible.


In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab import files
from pathlib import Path

def apply_terminator_grade(frame_bgr, t, add_edges=True, add_scanlines=True):
    img = frame_bgr.copy()
    h, w = img.shape[:2]

    b = img[:, :, 0].astype(np.float32) * 0.25
    g = img[:, :, 1].astype(np.float32) * 0.45
    r = img[:, :, 2].astype(np.float32) * 1.25 + 15.0
    img = np.clip(np.stack([b, g, r], axis=2), 0, 255).astype(np.uint8)

    if add_edges:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 80, 160)
        edges_bgr = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        img = cv2.addWeighted(img, 1.0, edges_bgr, 0.35, 0)

    if add_scanlines:
        spacing = 4
        img[::spacing, :, :] = (img[::spacing, :, :] * 0.55).astype(np.uint8)

        bar_y = int((t * 120) % max(1, h))
        y1, y2 = max(0, bar_y - 8), min(h, bar_y + 8)
        if y2 > y1:
            bar = img[y1:y2].astype(np.float32)
            bar[:, :, 2] = np.clip(bar[:, :, 2] * 1.35 + 20, 0, 255)
            img[y1:y2] = bar.astype(np.uint8)

    noise = np.random.randint(-8, 9, img.shape, dtype=np.int16)
    img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    yy, xx = np.mgrid[0:h, 0:w]
    cx, cy = w / 2.0, h / 2.0
    dist = np.sqrt(((xx - cx) / cx) ** 2 + ((yy - cy) / cy) ** 2)
    vign = np.clip(1.0 - 0.35 * dist, 0.65, 1.0).astype(np.float32)
    img = (img.astype(np.float32) * vign[:, :, None]).astype(np.uint8)
    return img

def draw_hud(img, dets, target, fps, frame_idx):
    out = img
    h, w = out.shape[:2]

    for x in range(0, w, 80):
        cv2.line(out, (x, 0), (x, h), (0, 0, 50), 1)
    for y in range(0, h, 60):
        cv2.line(out, (0, y), (w, y), (0, 0, 50), 1)

    cv2.putText(out, "CYBERDYNE VISION SYSTEM", (15, 28),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 230), 2, cv2.LINE_AA)
    cv2.putText(out, f"FPS:{fps:5.1f}  FRAME:{frame_idx}", (15, 54),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 200), 2, cv2.LINE_AA)

    for d in dets:
        (x1, y1, x2, y2) = d["xyxy"]
        is_target = target is not None and d is target

        color = (0, 0, 255) if is_target else (0, 0, 140)
        thickness = 3 if is_target else 1
        cv2.rectangle(out, (x1, y1), (x2, y2), color, thickness)

        tag = f'{d["cls_name"].upper()} {d["conf"]:.2f}'
        if is_target:
            tag += "  [TERMINATE]"

        (tw, th), _ = cv2.getTextSize(tag, cv2.FONT_HERSHEY_SIMPLEX, 0.55, 2)
        y_text = max(0, y1 - 8)
        cv2.rectangle(out, (x1, y_text - th - 6), (x1 + tw + 6, y_text + 4), (0, 0, 0), -1)
        cv2.putText(out, tag, (x1 + 3, y_text),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0, 0, 230), 2, cv2.LINE_AA)

    if target is not None:
        x1, y1, x2, y2 = target["xyxy"]
        cx, cy = (x1 + x2)//2, (y1 + y2)//2
        size = 18
        cv2.line(out, (cx - size, cy), (cx + size, cy), (0, 0, 255), 2)
        cv2.line(out, (cx, cy - size), (cx, cy + size), (0, 0, 255), 2)
        cv2.circle(out, (cx, cy), 28, (0, 0, 255), 2)

        status = f'TARGET:{target["cls_name"].upper()}  CONF:{target["conf"]:.2f}'
        cv2.putText(out, status, (15, h - 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 230), 2, cv2.LINE_AA)
        cv2.putText(out, "STATUS: LOCKED", (15, h - 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 230), 2, cv2.LINE_AA)
    else:
        cv2.putText(out, "STATUS: SEARCHING", (15, h - 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 200), 2, cv2.LINE_AA)

    return out

def run_video(in_path, out_path="out_terminator.mp4", model_name="yolov8n.pt",
              target_class="person", conf=0.25, iou=0.7, imgsz=640, max_frames=None):
    model = YOLO(model_name)

    cap = cv2.VideoCapture(str(in_path))
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open: {in_path}")

    fps_in = cap.get(cv2.CAP_PROP_FPS)
    if not fps_in or fps_in <= 0:
        fps_in = 30.0

    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(out_path, fourcc, fps_in, (w, h))

    frame_idx = 0
    t = 0.0
    fps_est = 0.0
    prev = cv2.getTickCount()

    while True:
        ok, frame = cap.read()
        if not ok:
            break

        # YOLO
        res = model.predict(source=frame, conf=conf, iou=iou, imgsz=imgsz, verbose=False)[0]
        dets = []
        if res.boxes is not None and len(res.boxes) > 0:
            xyxy = res.boxes.xyxy.cpu().numpy()
            confs = res.boxes.conf.cpu().numpy()
            clss = res.boxes.cls.cpu().numpy().astype(int)
            names = res.names
            for (x1, y1, x2, y2), c, cid in zip(xyxy, confs, clss):
                dets.append({
                    "xyxy": (int(x1), int(y1), int(x2), int(y2)),
                    "conf": float(c),
                    "cls_id": int(cid),
                    "cls_name": (names[cid] if isinstance(names, (list, dict)) else str(cid)) if names is not None else str(cid)
                })

        # pick target
        tc = target_class.strip().lower()
        candidates = [d for d in dets if d["cls_name"].lower() == tc]
        target = None
        if candidates:
            target = max(candidates, key=lambda d: ( (d["xyxy"][2]-d["xyxy"][0])*(d["xyxy"][3]-d["xyxy"][1]) * d["conf"] ))
        elif dets:
            target = max(dets, key=lambda d: d["conf"])

        # fps estimate
        now = cv2.getTickCount()
        dt = (now - prev) / cv2.getTickFrequency()
        prev = now
        if dt > 0:
            fps_est = 0.9 * fps_est + 0.1 * (1.0/dt) if fps_est > 0 else (1.0/dt)

        # effect + HUD
        t += 1.0 / fps_in
        stylized = apply_terminator_grade(frame, t)
        hud = draw_hud(stylized, dets, target, fps_est, frame_idx)

        writer.write(hud)
        frame_idx += 1
        if max_frames is not None and frame_idx >= max_frames:
            break

    cap.release()
    writer.release()
    return out_path


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
uploaded = files.upload()  # mp4等を選択
in_file = next(iter(uploaded.keys()))
out_file = run_video(in_file, out_path="out_terminator.mp4", target_class="person", model_name="yolov8n.pt")
out_file



Saving 36510-411342239_small.mp4 to 36510-411342239_small.mp4


'out_terminator.mp4'